In [5]:
import pandas as pd
import os
import ast
import json
import re

In [ ]:
# Carpeta donde están los archivos parquet
ruta = 'data'

# Lista todos los archivos parquet en la carpeta
archivos_parquet = [f for f in os.listdir(ruta) if f.endswith('.parquet')]

# Lee y concatena todos los archivos
dataframes = [pd.read_parquet(os.path.join(ruta, archivo)) for archivo in archivos_parquet]

# Une todos los dataframes en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Guarda el resultado en un nuevo archivo parquet 
df_final.to_parquet('bitacora_completa.parquet', index=False)

print(f"Se juntaron {len(archivos_parquet)} archivos en 'bitacora_completa.parquet'")


In [30]:
df = pd.read_parquet("data\AnalyticsRequestsXTracesLogs_2025-02-20.parquet")
df.head(3)

,OperationId,Url,TimeGenerated,TenantId,TimeGenerated1,Message,SeverityLevel,Properties,Measurements,OperationName,...,ClientBrowser,ResourceGUID,IKey,SDKVersion,ItemCount,ReferencedItemId,ReferencedType,SourceSystem,Type,_ResourceId
0,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:59:58.4533578Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,NaN,b7093352-752b-4052-ada3-2a30793d7f01,5cdf9557-7f12-4b1d-9614-eff2d7e1fae9,log4net:2.18.0-315,319,NaN,NaN,Azure,AppTraces,/subscriptions/4166267b-742e-48cb-92f0-f42e47f...
1,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:58:58.45184Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,NaN,b7093352-752b-4052-ada3-2a30793d7f01,5cdf9557-7f12-4b1d-9614-eff2d7e1fae9,log4net:2.18.0-315,319,NaN,NaN,Azure,AppTraces,/subscriptions/4166267b-742e-48cb-92f0-f42e47f...
2,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:57:58.4539896Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,NaN,b7093352-752b-4052-ada3-2a30793d7f01,5cdf9557-7f12-4b1d-9614-eff2d7e1fae9,log4net:2.18.0-315,319,NaN,NaN,Azure,AppTraces,/subscriptions/4166267b-742e-48cb-92f0-f42e47f...


In [31]:
df["Properties"].head(10)

0    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
1    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
2    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
3    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
4    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
5    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
6    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
7    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
8    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
9    {"LineNumber":"0","SessionId":"0zwolsr3yalmdcp...
Name: Properties, dtype: object

In [ ]:
def extract_properties(s):
    if pd.isnull(s):
        return {}
    return dict(re.findall(r'"([^"]+)":"([^"]*)"', s))

df['Properties_dict'] = df['Properties'].apply(extract_properties)

properties_df = df['Properties_dict'].apply(pd.Series)
properties_df = properties_df.add_prefix('Prop_')

df = pd.concat([df, properties_df], axis=1)

df = df.drop(columns=['Properties_dict'])

df.head()

,OperationId,Url,TimeGenerated,TenantId,TimeGenerated1,Message,SeverityLevel,Properties,Measurements,OperationName,...,Prop_SessionId,Prop_UserId,Prop_AppId,Prop_MethodName,Prop_ClassName,Prop_LogFileId,Prop_ThreadName,Prop_LoggerName,Prop_SiteId,Prop_Domain
0,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:59:58.4533578Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,0zwolsr3yalmdcpnnoj1uzl1,ca4921038ebe186bf5a990372bd8f41a1f1441180f2fcf...,a75f08e5-c03b-4511-8d61-348949929587,ProcessBRRequest,_Default,20250521_43816,77,AppLog,9c56cf49-f62a-4233-8777-133b9286463a,/LM/W3SVC/745662481/ROOT-1-133922898906471714
1,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:58:58.45184Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,0zwolsr3yalmdcpnnoj1uzl1,ca4921038ebe186bf5a990372bd8f41a1f1441180f2fcf...,a75f08e5-c03b-4511-8d61-348949929587,ProcessBRRequest,_Default,20250521_43816,77,AppLog,9c56cf49-f62a-4233-8777-133b9286463a,/LM/W3SVC/745662481/ROOT-1-133922898906471714
2,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:57:58.4539896Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,0zwolsr3yalmdcpnnoj1uzl1,ca4921038ebe186bf5a990372bd8f41a1f1441180f2fcf...,a75f08e5-c03b-4511-8d61-348949929587,ProcessBRRequest,_Default,20250521_43816,77,AppLog,9c56cf49-f62a-4233-8777-133b9286463a,/LM/W3SVC/745662481/ROOT-1-133922898906471714
3,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:56:58.7410508Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,0zwolsr3yalmdcpnnoj1uzl1,ca4921038ebe186bf5a990372bd8f41a1f1441180f2fcf...,a75f08e5-c03b-4511-8d61-348949929587,ProcessBRRequest,_Default,20250521_43816,77,AppLog,9c56cf49-f62a-4233-8777-133b9286463a,/LM/W3SVC/745662481/ROOT-1-133922898906471714
4,b639d78b3bdc48818e782bd948cf413e,https://pms3.innsist.tca.com/Default.aspx/BRRe...,2025-05-22T00:55:58.461369Z,64c42171-d056-4356-9cdd-9f3cee0146e5,2025-05-22T00:18:16.6747026Z,WebMethod=ProcessBRRequest | ReqId:328,1,"{""LineNumber"":""0"",""SessionId"":""0zwolsr3yalmdcp...",NaN,POST /Default.aspx/BRRequest,...,0zwolsr3yalmdcpnnoj1uzl1,ca4921038ebe186bf5a990372bd8f41a1f1441180f2fcf...,a75f08e5-c03b-4511-8d61-348949929587,ProcessBRRequest,_Default,20250521_43816,77,AppLog,9c56cf49-f62a-4233-8777-133b9286463a,/LM/W3SVC/745662481/ROOT-1-133922898906471714


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65795 entries, 0 to 65794
Data columns (total 48 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   OperationId            65795 non-null  object 
 1   Url                    65795 non-null  object 
 2   TimeGenerated          65795 non-null  object 
 3   TenantId               65795 non-null  object 
 4   TimeGenerated1         65795 non-null  object 
 5   Message                65795 non-null  object 
 6   SeverityLevel          65795 non-null  int64  
 7   Properties             65795 non-null  object 
 8   Measurements           0 non-null      float64
 9   OperationName          65790 non-null  object 
 10  OperationId1           65795 non-null  object 
 11  ParentId               65795 non-null  object 
 12  SyntheticSource        0 non-null      float64
 13  SessionId              36734 non-null  object 
 14  UserId                 65790 non-null  object 
 15  Us

In [ ]:
#Quitar columnas con un solo valor
uninformative_cols = [col for col in df.columns if df[col].nunique(dropna=False) == 1]
print(f"Uninformative columns: {uninformative_cols}")
df = df.drop(columns=uninformative_cols)
df.info()

Uninformative columns: ['TenantId', 'Measurements', 'SyntheticSource', 'UserAuthenticatedId', 'UserAccountId', 'AppVersion', 'AppRoleName', 'ClientType', 'ClientModel', 'ClientOS', 'ClientIP', 'ClientBrowser', 'ResourceGUID', 'IKey', 'SDKVersion', 'ReferencedItemId', 'ReferencedType', 'SourceSystem', 'Type', '_ResourceId', 'Prop_LineNumber', 'Prop_SiteId']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65795 entries, 0 to 65794
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   OperationId            65795 non-null  object
 1   Url                    65795 non-null  object
 2   TimeGenerated          65795 non-null  object
 3   TimeGenerated1         65795 non-null  object
 4   Message                65795 non-null  object
 5   SeverityLevel          65795 non-null  int64 
 6   Properties             65795 non-null  object
 7   OperationName          65790 non-null  object
 8   OperationId1     

In [ ]:
def extract_properties(s):
    if pd.isnull(s):
        return {}
    return dict(re.findall(r'"([^"]+)":"([^"]*)"', s))

ruta = 'data'
archivos_parquet = [f for f in os.listdir(ruta) if f.endswith('.parquet')]

# 1. ORDENAR LOS ARCHIVOS:
archivos_parquet.sort() 

# 2. CREAR EL RANGO DE FECHAS
fechas = pd.date_range(start="2025-02-20", end="2025-05-19", freq='D')
dataframes = []

for i, archivo in enumerate(archivos_parquet):
    # Asigna la fecha correspondiente usando el índice del bucle
    fecha_actual = fechas[i]
    
    df = pd.read_parquet(os.path.join(ruta, archivo))
    
    # Agregar la columna de fecha 
    df['fecha'] = fecha_actual
    df['Properties_dict'] = df['Properties'].apply(extract_properties)
    properties_df = df['Properties_dict'].apply(pd.Series).add_prefix('Prop_')
    df = pd.concat([df, properties_df], axis=1)
    df = df.drop(columns=['Properties_dict'])
    dataframes.append(df)
    print(f"[{i+1}/{len(archivos_parquet)}] Procesado: {archivo} --> Asignada fecha: {fecha_actual.strftime('%Y-%m-%d')}")
print("\nConcatenando todos los archivos...")
df_final = pd.concat(dataframes, ignore_index=True)

# Remove columns where all values are the same
uninformative_cols = [col for col in df_final.columns if df_final[col].nunique(dropna=False) == 1]
if uninformative_cols:
    print(f"Eliminando columnas con un único valor: {uninformative_cols}")
    df_final = df_final.drop(columns=uninformative_cols)

# Save the result
print("Guardando el archivo final...")
df_final.to_parquet('bitacora_completa3.parquet', index=False)

[1/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-20.parquet --> Asignada fecha: 2025-02-20
[2/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-21.parquet --> Asignada fecha: 2025-02-21
[3/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-22.parquet --> Asignada fecha: 2025-02-22
[4/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-23.parquet --> Asignada fecha: 2025-02-23
[5/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-24.parquet --> Asignada fecha: 2025-02-24
[6/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-25.parquet --> Asignada fecha: 2025-02-25
[7/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-26.parquet --> Asignada fecha: 2025-02-26
[8/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-27.parquet --> Asignada fecha: 2025-02-27
[9/89] Procesado: AnalyticsRequestsXTracesLogs_2025-02-28.parquet --> Asignada fecha: 2025-02-28
[10/89] Procesado: AnalyticsRequestsXTracesLogs_2025-03-01.parquet --> Asignada fecha: 2025-03-01
[11/89] Procesado: AnalyticsR

In [12]:
df_final = pd.read_parquet('2bitacora_completa.parquet')
df_final.head()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5780399 entries, 0 to 5780398
Data columns (total 26 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   OperationId            object
 1   Url                    object
 2   TimeGenerated          object
 3   TimeGenerated1         object
 4   Message                object
 5   SeverityLevel          int64 
 6   Properties             object
 7   OperationName          object
 8   OperationId1           object
 9   ParentId               object
 10  SessionId              object
 11  UserId                 object
 12  AppRoleInstance        object
 13  ClientCity             object
 14  ClientStateOrProvince  object
 15  ClientCountryOrRegion  object
 16  ItemCount              int64 
 17  Prop_SessionId         object
 18  Prop_UserId            object
 19  Prop_AppId             object
 20  Prop_MethodName        object
 21  Prop_ClassName         object
 22  Prop_LogFileId         object
 23  Prop_Th